# 第七章　給湯設備

## 第一節　給湯設備

1．適用範囲～4．記号及び単位は省略

### 5. 給湯設備によるエネルギー消費量

#### 5.1 消費電力量

台所、洗面所及び浴室等がいずれも無い場合は0とする。 

$$\large
\displaystyle E_{E,W,d,t} = E_{E,hs,d,t} + E_{E,aux,ss,d,t}
$$

$ E_{E,W,d,t} $：日付$d$の時刻$t$における 1時間当たりの給湯設備の消費電力量 (kWh/h)  
$ E_{E,aux,ss,d,t} $：日付$d$の時刻$t$における 1時間当たりの太陽熱利用設備の補機の消費電力量 (kWh/h)    
$ E_{E,hs,d,t} $：日付$d$の時刻$t$における1時間当たりの給湯機（給湯温水暖房機における給湯部も含む。以下、同じ。）の消費電力量 (kWh/h)  

日付$d$の時刻$t$における1時間当たりの太陽熱利用設備の補機の消費電力量$ E_{E,aux,ss,d,t} $は、液体集熱式太陽熱利用設備及び空気集熱式太陽熱利用設備のどちらも採用しない場合、ゼロとする。

#### 5.2 ガス消費量

台所、洗面所及び浴室等がいずれも無い場合は0とする。 

$$\large
\displaystyle E_{G,W,d,t} = E_{G,hs,d,t}
$$

$ E_{G,W,d,t} $：日付$d$の時刻$t$における1時間当たりの給湯設備のガス消費量 (MJ/h)  
$ E_{G,hs,d,t} $：日付$d$の時刻$t$における1時間当たりの給湯機のガス消費量 (MJ/h)  

#### 5.3 灯油消費量

台所、洗面所及び浴室等がいずれも無い場合は0とする。 

$$\large
\displaystyle E_{K,W,d,t} = E_{K,hs,d,t}
$$

$ E_{K,W,d,t} $：日付$d$の時刻$t$における1時間当たりの給湯設備の灯油消費量 (MJ/h)  
$ E_{K,hs,d,t} $：日付$d$の時刻$t$における1時間当たりの給湯機の灯油消費量 (MJ/h)  

#### 5.4 その他の燃料による一次エネルギー消費量

日付$d$の時刻$t$における1時間当たりの給湯設備のその他の燃料による一次エネルギー消費量$E_{M,w,d,t}$は0とする。 

### 6. 給湯機のエネルギー消費量

省略

### 7. 太陽熱補正給湯熱負荷

$$\large
\displaystyle
L''_{k,d,t} = L'_{k,d,t} - L_{sun,d,t} \times \frac{L'_{k,d,t}}{(L'_{k,d,t}+L'_{s,d,t}+L'_{w,d,t}+L'_{b1,d,t}+L'_{b2,d,t}+L'_{ba1,d,t})}
$$


$$\large
\displaystyle L''_{s,d,t} = L'_{s,d,t} - L_{sun,d,t} \times \frac { L'_{s,d,t}}{ (L'_{k,d,t} + L'_{s,d,t} + L'_{w,d,t} + L'_{b1,d,t} + L'_{b2,d,t} + L'_{ba1,d,t} ) }
$$


$$\large
\displaystyle L''_{w,d,t} = L'_{w,d,t} - L_{sun,d,t} \times \frac { L'_{w,d,t}}{ (L'_{k,d,t} + L'_{s,d,t} + L'_{w,d,t} + L'_{b1,d,t} + L'_{b2,d,t} + L'_{ba1,d,t} ) }
$$  


$$\large
\displaystyle L''_{b1,d,t}= L'_{b1,d,t} - L_{sun,d,t} \times \frac{ L'_{b1,d,t}} {(L'_{k,d,t} + L'_{s,d,t} + L'_{w,d,t} + L'_{b1,d,t} + L'_{b2,d,t} + L'_{ba1,d,t} ) }
$$


$$\large
\displaystyle L''_{b2,d,t}= L'_{b2,d,t} - L_{sun,d,t} \times \frac{ L'_{b2,d,t}} {(L'_{k,d,t} + L'_{s,d,t} + L'_{w,d,t} + L'_{b1,d,t} + L'_{b2,d,t} + L'_{ba1,d,t} ) }
$$


$$\large
\displaystyle L''_{ba1,d,t} = L'_{ba1,d,t} - L_{sun,d,t} \times \frac{L'_{ba1,d,t}}{(L'_{k,d,t} + L'_{s,d,t} + L'_{w,d,t} + L'_{b1,d,t} + L'_{b2,d,t} + L'_{ba1,d,t})}
$$

$$\large
\displaystyle L''_{ba2,d,t} = L'_{ba2,d,t}
$$

$ L''_{k,d,t} $：日付$d$の時刻$t$における1時間当たりの台所水栓における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{s,d,t} $：日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{w,d,t} $：日付$d$の時刻$t$における1時間当たりの洗面水栓における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{b1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{b2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽自動湯はり時における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{ba1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷(MJ/h)  
$ L''_{ba2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽追焚時における太陽熱補正給湯熱負荷(MJ/h)  
$ L'_{k,d,t} $：日付$d$の時刻$t$における1時間当たりの台所水栓における節湯補正給湯熱負荷(MJ/h)  
$ L'_{s,d,t} $：日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における節湯補正給湯熱負荷(MJ/h)  
$ L'_{w,d,t} $：日付$d$の時刻$t$における1時間当たりの洗面水栓における節湯補正給湯熱負荷(MJ/h)  
$ L'_{b1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷(MJ/h)  
$ L'_{b2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽自動湯はり時における節湯補正給湯熱負荷(MJ/h)  
$ L'_{ba1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷(MJ/h)  
$ L'_{ba2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽追焚時における節湯補正給湯熱負荷(MJ/h)  
$ L_{sun,d,t} $：日付$d$の時刻$t$における1時間当たりの太陽熱利用給湯設備による補正集熱量(MJ/h)

日付$d$の時刻$t$における1時間当たりの太陽熱利用給湯設備による補正集熱量$ L_{sun,d,t} $は、液体集熱式太陽熱利用設備及び空気集熱式太陽熱利用設備のどちらも採用しない場合、ゼロとする。

### 8. 節湯補正給湯熱負荷

$$\large
\displaystyle L'_{k,d,t} = W'_{k,d,t} \times (\theta_{sw,k} - \theta_{wtr,d}) \times 4.186 \times 10^{-3}
$$

$$\large
\displaystyle L'_{s,d,t} = W'_{s,d,t} \times (\theta_{sw,s} - \theta_{wtr,d}) \times 4.186 \times 10^{-3}
$$

$$\large
\displaystyle L'_{w,d,t} = W'_{w,d,t} \times (\theta_{sw,w} - \theta_{wtr,d}) \times 4.186 \times 10^{-3}
$$

浴室等が無い場合（ふろ機能の種類によらない）

$$\large
\displaystyle L'_{b1,d,t} = 0.0
$$

$$\large
\displaystyle L'_{b2,d,t} = 0.0
$$

浴室等が有り、ふろ機能の種類が給湯単機能の場合 

$$\large
\displaystyle L'_{b1,d,t} = W'_{b1,d,t} \times ( \theta_{sw,b1} - \theta_{wtr,d} ) \times 4.186 \times 10^{-3}
$$

$$\large
\displaystyle L'_{b2,d,t} = 0.0
$$

浴室等が有り、ふろ機能の種類がふろ給湯機（追焚の有無によらない）の場合

$$\large
\displaystyle L'_{b1,d,t} = 0.0
$$

$$\large
\displaystyle L'_{b2,d,t} = W'_{b2,d,t} \times ( \theta_{sw,b2} - \theta_{wtr,d} ) \times 4.186 \times 10^{-3}
$$

浴室等が無い場合

$$\large
\displaystyle L'_{ba1,d,t} = 0.0
$$

$$\large
\displaystyle L'_{ba2,d,t} = 0.0
$$

浴室等が有り、ふろ機能の種類が給湯単機能またはふろ給湯機（追焚なし）の場合 

$$\large
\displaystyle L'_{ba1,d,t} = W'_{ba1,d,t} \times ( \theta_{sw,ba1} - \theta_{wtr,d} ) \times 4.186 \times 10^{-3}
$$

$$\large
\displaystyle L'_{ba2,d,t} = 0.0
$$

浴室等が有り、ふろ機能の種類がふろ給湯機（追焚あり）の場合

$$\large
\displaystyle L'_{ba1,d,t} = 0.0
$$

$$\large
\displaystyle L'_{ba2,d,t} = L_{ba,d,t} \times 1.25
$$

$ L'_{k,d,t} $：日付$d$の時刻$t$における1時間当たりの台所水栓における節湯補正給湯熱負荷(MJ/h)  
$ L'_{s,d,t} $：日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における節湯補正給湯熱負荷(MJ/h)  
$ L'_{w,d,t} $：日付$d$の時刻$t$における1時間当たりの洗面水栓における節湯補正給湯熱負荷(MJ/h)  
$ L'_{b1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷(MJ/h)  
$ L'_{b2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽自動湯はり時における節湯補正給湯熱負荷(MJ/h)  
$ L'_{ba1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷(MJ/h)  
$ L'_{ba2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽追焚時における節湯補正給湯熱負荷(MJ/h)  
$ W'_{k,d,t} $：日付$d$の時刻$t$における1時間当たりの台所水栓における節湯補正給湯量(L/h)  
$ W'_{s,d,t} $：日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における節湯補正給湯量(L/h)  
$ W'_{w,d,t} $：日付$d$の時刻$t$における1時間当たりの洗面水栓における節湯補正給湯量(L/h)  
$ W'_{b1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓湯はり時における節湯補正給湯量(L/h)  
$ W'_{b2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽自動湯はり時における節湯補正給湯量(L/h)  
$ W'_{ba1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における節湯補正給湯量(L/h)  
$ W'_{ba2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽追焚時における節湯補正給湯量(L/h)  
$ L_{ba,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽沸かし直しによる給湯熱負荷(MJ/h)  
$ \theta_{sw,k} $：台所水栓における基準給湯温度(℃)  
$ \theta_{sw,s} $：浴室シャワー水栓における基準給湯温度(℃)  
$ \theta_{sw,w} $：洗面水栓における基準給湯温度(℃)  
$ \theta_{sw,b1} $：浴槽水栓湯はり時における基準給湯温度(℃)  
$ \theta_{sw,b2} $：浴槽自動湯はり時における基準給湯温度(℃)  
$ \theta_{sw,ba1} $：浴槽水栓さし湯時における基準給湯温度(℃)  
$ \theta_{sw,ba2} $：浴槽追焚時における基準給湯温度(℃)  
$ \theta_{wtr,d} $：日付$d$における日平均給水温度(℃)

<div style="text-align: center;font-weight: bold;">表　用途ごとの基準給湯温度</div>

| 用途 |　記号　| 基準給湯温度(℃) |
| ----- | ---- | ---- |
| 台所水栓 | $\theta_{sw,k}$ | 40 |
|浴室シャワー水栓|$\theta_{sw,s}$|40|
|洗面水栓|$\theta_{sw_w}$|40|
|浴槽水栓湯はり|$\theta_{sw,b1}$|40|
|浴槽自動湯はり|$\theta_{sw,b1}$|40|
|浴槽水栓さし湯|$\theta_{sw,ba1}$|60|


### 9. 節湯補正給湯量

$$\large
\displaystyle W'_{k,d,t} = W_{k,d,t} \times f_{sk} \times f_{sp}
$$

$$\large
\displaystyle W'_{s,d,t} = W_{s,d,t} \times f_{ss} \times f_{sp}
$$

$$\large
\displaystyle W'_{w,d,t} = W_{w,d,t} \times f_{sw} \times f_{sp}
$$

$$\large
\displaystyle W'_{b1,d,t} = W_{b1,d,t} \times f_{sp} \times f_{sb}
$$

$$\large
\displaystyle W'_{b2,d,t} = W_{b2,d,t} \times f_{sb}
$$

$$\large
\displaystyle W'_{ba1,d,t} = W_{ba1,d,t} \times f_{sp}
$$

$ W'_{k,d,t} $：日付$d$の時刻$t$における1時間当たりの台所水栓における節湯補正給湯量(L/h)  
$ W'_{s,d,t} $：日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における節湯補正給湯量(L/h)  
$ W'_{w,d,t} $：日付$d$の時刻$t$における1時間当たりの洗面水栓における節湯補正給湯量(L/h)  
$ W'_{b1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓湯はり時における節湯補正給湯量(L/h)  
$ W'_{b2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽自動湯はり時における節湯補正給湯量(L/h)  
$ W'_{ba1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における節湯補正給湯量(L/h)  
$ W'_{ba2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽追焚時における節湯補正給湯量(L/h)  
$ W_{k,d,t} $：日付$d$の時刻$t$における1時間当たりの台所水栓における基準給湯量(L/h)  
$ W_{s,d,t} $：日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における基準給湯量(L/h)  
$ W_{w,d,t} $：日付$d$の時刻$t$における1時間当たりの洗面水栓における基準給湯量(L/h)  
$ W_{b1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓湯はり時における基準給湯量(L/h)  
$ W_{b2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽自動湯はり時における基準給湯量(L/h)  
$ W_{ba1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における基準給湯量(L/h)  
$ W_{ba2,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽追焚時における基準給湯量(L/h)  
$ f_{sk} $：台所水栓における節湯の効果係数(-)  
$ f_{ss} $：浴室シャワー水栓における節湯の効果係数(-)  
$ f_{sw} $：洗面水栓における節湯の効果係数(-)  
$ f_{sp} $：配管における節湯の効果係数(-)  
$ f_{sb} $：浴槽における節湯の効果係数(-)  

### 10. 基準給湯量

$$\large
\displaystyle
W_{k,d,t} = \left\{ \begin{array}{ll}
    W_{k,p,d,t}|_{p=1} \times \displaystyle \frac{2 - n_p}{2 - 1} + W_{k,p,d,t}|_{p=2} \displaystyle \times \frac{n_p - 1}{2 - 1} & (1 \leq n_p < 2)\\ 
    W_{k,p,d,t}|_{p=2} \times \displaystyle \frac{3 - n_p}{3 - 2} + W_{k,p,d,t}|_{p=3} \displaystyle \times \frac{n_p - 2}{3 - 2} & (2 \leq n_p < 3)\\ 
    W_{k,p,d,t}|_{p=3} \times \displaystyle \frac{4 - n_p}{4 - 3} + W_{k,p,d,t}|_{p=4} \displaystyle \times \frac{n_p - 3}{4 - 3} & (3 \leq n_p \leq 4)\\ 
    \end{array}
\right.
$$


$$\large
\displaystyle
W_{s,d,t} = \left\{ \begin{array}{ll}
    W_{s,p,d,t}|_{p=1} \times \displaystyle \frac{2 - n_p}{2 - 1} + W_{s,p,d,t}|_{p=2} \displaystyle \times \frac{n_p - 1}{2 - 1} & (1 \leq n_p < 2)\\ 
    W_{s,p,d,t}|_{p=2} \times \displaystyle \frac{3 - n_p}{3 - 2} + W_{s,p,d,t}|_{p=3} \displaystyle \times \frac{n_p - 2}{3 - 2} & (2 \leq n_p < 3)\\ 
    W_{s,p,d,t}|_{p=3} \times \displaystyle \frac{4 - n_p}{4 - 3} + W_{s,p,d,t}|_{p=4} \displaystyle \times \frac{n_p - 3}{4 - 3} & (3 \leq n_p \leq 4)\\ 
    \end{array}
\right.
$$


$$\large
\displaystyle
W_{w,d,t} = \left\{ \begin{array}{ll}
    W_{w,p,d,t}|_{p=1} \times \displaystyle \frac{2 - n_p}{2 - 1} + W_{w,p,d,t}|_{p=2} \displaystyle \times \frac{n_p - 1}{2 - 1} & (1 \leq n_p < 2)\\ 
    W_{w,p,d,t}|_{p=2} \times \displaystyle \frac{3 - n_p}{3 - 2} + W_{w,p,d,t}|_{p=3} \displaystyle \times \frac{n_p - 2}{3 - 2} & (2 \leq n_p < 3)\\ 
    W_{w,p,d,t}|_{p=3} \times \displaystyle \frac{4 - n_p}{4 - 3} + W_{w,p,d,t}|_{p=4} \displaystyle \times \frac{n_p - 3}{4 - 3} & (3 \leq n_p \leq 4)\\ 
    \end{array}
\right.
$$

$ n_{p} $：仮想居住人数(-)  
$ W_{k,d,t} $：日付$d$の時刻$t$における1時間当たりの台所水栓における基準給湯量(L/h)  
$ W_{k,p,d,t} $：日付$d$の時刻$t$における1時間当たりの居住人数が$p$人における台所水栓における基準給湯量(L/h)  
$ W_{s,d,t} $：日付$d$の時刻$t$における1時間当たりの浴室シャワー水栓における基準給湯量(L/h)  
$ W_{s,p,d,t} $：日付$d$の時刻$t$における1時間当たりの居住人数が$p$人における浴室シャワー水栓における基準給湯量(L/h)  
$ W_{w,d,t} $：日付$d$の時刻$t$における1時間当たりの洗面水栓における基準給湯量(L/h)  
$ W_{w,p,d,t} $：日付$d$の時刻$t$における1時間当たりの居住人数が$p$人における洗面水栓における基準給湯量(L/h)  

$$\large
\displaystyle
W_{b,d,t} = \left\{ \begin{array}{ll}
    W_{b,p,d,t}|_{p=1} \times \displaystyle \frac{2 - n_p}{2 - 1} + W_{b,p,d,t}|_{p=2} \displaystyle \times \frac{n_p - 1}{2 - 1} & (1 \leq n_p < 2)\\ 
    W_{b,p,d,t}|_{p=2} \times \displaystyle \frac{3 - n_p}{3 - 2} + W_{b,p,d,t}|_{p=3} \displaystyle \times \frac{n_p - 2}{3 - 2} & (2 \leq n_p < 3)\\ 
    W_{b,p,d,t}|_{p=3} \times \displaystyle \frac{4 - n_p}{4 - 3} + W_{b,p,d,t}|_{p=4} \displaystyle \times \frac{n_p - 3}{4 - 3} & (3 \leq n_p \leq 4)\\ 
    \end{array}
\right.
$$

$ n_{p} $：仮想居住人数(-)  
$ W_{b,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽湯はり時における基準給湯量(L/h)  
$ W_{b,p,d,t} $：日付$d$の時刻$t$における1時間当たりの居住人数が$p$人における浴槽湯はり時における基準給湯量(L/h)  

日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における基準給湯量$ W_{ba1,d,t} $は、ふろ機能の種類が給湯単機能又はふろ給湯機（追焚なし）の場合のみに発生する。

$$\large
\displaystyle W_{ba1,d,t} = L_{ba,d,t} \times \frac{1}{(\theta_{sw,ba1} - \theta_{wtr,d})} \times \frac{1}{4.186} \times 10^3
$$

$ L_{ba,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽沸かし直しによる給湯熱負荷(MJ/h)  
$ W_{ba1,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽水栓さし湯時における基準給湯量(L/h)  
$ \theta_{sw,ba1} $：浴槽水栓さし湯時における基準給湯温度(℃)  
$ \theta_{wtr,d} $：日付$d$における日平均給水温度(℃)  

### 11. 浴槽沸かし直しによる給湯熱負荷

$$\large
\displaystyle
L_{ba,d,t} = \left\{ \begin{array}{ll}
    L_{ba,p,d,t}|_{p=1} \times \displaystyle \frac{2 - n_p}{2 - 1} + L_{ba,p,d,t}|_{p=2} \times \displaystyle \frac{n_p - 1}{2 - 1} & (1 \leq n_p < 2)\\ 
    L_{ba,p,d,t}|_{p=2} \times \displaystyle \frac{3 - n_p}{3 - 2} + L_{ba,p,d,t}|_{p=3} \times \displaystyle \frac{n_p - 2}{3 - 2} & (2 \leq n_p < 3)\\ 
    L_{ba,p,d,t}|_{p=3} \times \displaystyle \frac{4 - n_p}{4 - 3} + L_{ba,p,d,t}|_{p=4} \times \displaystyle \frac{n_p - 3}{4 - 3} & (3 \leq n_p \leq 4)\\ 
    \end{array}
\right.
$$


$ n_{p} $：仮想居住人数(-)  
$ L_{ba,d,t} $：日付$d$の時刻$t$における1時間当たりの浴槽沸かし直しによる給湯熱負荷(MJ/h)  
$ L_{ba,p,d,t} $：日付$d$の時刻$t$における1時間当たりの居住人数が$p$人における浴槽沸かし直しによる給湯熱負荷(MJ/h)  

$$\large
\displaystyle
L_{ba,p,d,t} = \left\{ \begin{array}{ll}
    \big(a_{ba,p,d} \times \theta_{ex,Ave,d} + b_{ba,p,d} \big) \times \frac{n_{b,p,d,t}}{\sum_{t=0}^{23} n_{b,p,d,t}}  & \bigg(\sum_{t=0}^{23} n_{b,p,d,t} > 0 \bigg)\\
    0 & \bigg(\sum_{t=0}^{23} n_{b,p,d,t} = 0 \bigg)\\
    \end{array}
\right.
$$

$L_{ba,p,d,t}$：日付$d$の時刻$t$における1時間当たりの居住人数が$p$人における浴槽沸かし直しによる給湯熱負荷(MJ/h)  
$n_{b,p,d,t}$：日付$d$の時刻$t$における1時間当たりの居住人数が$p$人における浴槽入浴人数(人/h)  
$W_{b,p,d,t}$：日付$d$の時刻$t$における1時間当たりの居住人数が$p$人における浴槽湯はり時における基準給湯量(L/h)  
$\theta_{ex,Ave,d}$：日付$d$における日平均外気温度(℃)  
$a_{ba,p,d}, b_{ba,p,d}$：係数(-)  

<div style="text-align: center;font-weight: bold;">表　係数 $a_{ba}$ 及び $b_{ba}$ の値</div>

<table>
<thead>
<tr>
<th rowspan="2">日平均外気温度の条件</th>
<th rowspan="2">浴槽の種類</th>
<th colspan="2" style="text-align:center;">4 人</th>
<th colspan="2" style="text-align:center;">3 人</th>
<th colspan="2" style="text-align:center;">2 人</th>
<th colspan="2" style="text-align:center;">1 人</th>
</tr>
<tr>
<th>係数 $a_{ba}$</th>
<th>係数 $b_{ba}$</th>
<th>係数 $a_{ba}$</th>
<th>係数 $b_{ba}$</th>
<th>係数 $a_{ba}$</th>
<th>係数 $b_{ba}$</th>
<th>係数 $a_{ba}$</th>
<th>係数 $b_{ba}$</th>
</tr>
</thead>

<tbody>
<tr><td rowspan="2">7℃未満</td><td>通常浴槽</td><td>-0.12</td><td>6.00</td><td>-0.10</td><td>4.91</td><td>-0.06</td><td>3.02</td><td>0.00</td><td>0.00</td></tr>
<tr><td>高断熱浴槽</td><td>-0.07</td><td>3.98</td><td>-0.06</td><td>3.22</td><td>-0.04</td><td>2.01</td><td>0.00</td><td>0.00</td></tr>
<tr><td rowspan="2">7℃以上かつ16℃未満</td><td>通常浴槽</td><td>-0.13</td><td>6.04</td><td>-0.10</td><td>4.93</td><td>-0.06</td><td>3.04</td><td>0.00</td><td>0.00</td></tr>
<tr><td>高断熱浴槽</td><td>-0.08</td><td>4.02</td><td>-0.06</td><td>3.25</td><td>-0.04</td><td>2.03</td><td>0.00</td><td>0.00</td></tr>
<tr><td rowspan="2">16℃以上かつ25℃未満</td><td>通常浴槽</td><td>-0.14</td><td>6.21</td><td>-0.11</td><td>5.07</td><td>-0.07</td><td>3.13</td><td>0.00</td><td>0.00</td></tr>
<tr><td>高断熱浴槽</td><td>-0.09</td><td>4.19</td><td>-0.07</td><td>3.39</td><td>-0.04</td><td>2.12</td><td>0.00</td><td>0.00</td></tr>
<tr><td rowspan="2">25℃以上</td><td>通常浴槽</td><td>-0.12</td><td>5.81</td><td>-0.10</td><td>4.77</td><td>-0.06</td><td>2.92</td><td>0.00</td><td>0.00</td></tr>
<tr><td>高断熱浴槽</td><td>-0.07</td><td>3.80</td><td>-0.06</td><td>3.09</td><td>-0.04</td><td>1.92</td><td>0.00</td><td>0.00</td></tr>
</tbody>

</table>


### 12. 日平均給水温度

$$\large 
\theta_{wtr,d} = max(a_{wtr} \times \theta_{ex,prd,Ave,d} + b_{wtr}, 0.5)
$$

$ \theta_{ex,prd,Ave,d}$：日付$d$を基準とした期間平均外気温度(℃)  
$ \theta_{wtr,d}$：日付$d$における日平均給水温度(℃)  
$ a_{wtr}, b_{wtr} $：日平均給水温度を求める際の回帰係数  

<div style="text-align: center;font-weight: bold;">表　日平均給水温度を求める際の回帰係数の値</div>

|地域の区分|$a_{wtr}$|$b_{wtr}$|
|-|-|-|
|1|0.6639|3.466|
|2|0.6639|3.466|
|3|0.6054|4.515|
|4|0.6054|4.515|
|5|0.8660|1.665|
|6|0.8516|2.473|
|7|0.9223|2.097|
|8|0.6921|7.167|


$$\large 
\theta_{ex,prd,Ave,d} = \sum^{10}_{k = 1} \theta_{ex,Ave,d-k} / 10
$$

$\theta_{ex,Ave,d-k}$：日付$d$を基準とした$k$日前の日平均外気温度(℃)  
$\theta_{ex,prd,Ave,d}$：日付$d$を基準とした期間平均外気温度(℃)  

### 13. 日平均外気温度

$$\large 
\theta_{ex,Ave,d} = \sum^{23}_{t = 0} \theta_{ex,d,t} \big/ 24
$$

$ \theta_{ex,Ave,d} $：日付$d$における日平均外気温度(℃)  
$ \theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃)  

### 14. 夜間平均外気温度

$$\large 
\theta_{ex,Nave,d} = \bigg\{ \theta_{ex,d-1,23} + \sum^6_{t=0}\theta_{ex,d,t} \bigg\} \big/8
$$

$ \theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃)  
$ \theta_{ex,d-1,23} $：日付$d-1$における23 時における外気温度(℃)  
$ \theta_{ex,Nave,d}$：日付$d$における夜間平均外気温度(℃)  

### 15. 温水暖房の熱負荷

省略

## 付録 A 複数の給湯機を設置する場合の評価において代表させる給湯機の決定方法 

省略

## 付録 B 給湯機を設置しない場合に評価において想定する機器の種類と仕様 

省略